In [1]:
from dotenv import load_dotenv
load_dotenv('../.env') # replace with your .env file

True

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Telco.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Customers LIMIT 10;")

sqlite
['Billing', 'Complaints', 'Customers', 'Devices', 'Employees', 'Locations', 'Messages', 'Notifications', 'Orders', 'Payments', 'Plans', 'Repairs', 'Services', 'Subscriptions', 'Usage']


"[(1, 'John Doe', '123 Main St, Anytown, USA', '+1234567890'), (2, 'Alice Smith', '456 Elm St, Otherville, USA', '+1987654321'), (3, 'Bob Johnson', '789 Oak St, Anycity, USA', '+1122334455'), (4, 'Emily Brown', '321 Pine St, Newville, USA', '+1555666777'), (5, 'Michael Wilson', '654 Maple St, Anothercity, USA', '+1444222111'), (6, 'Sarah Davis', '987 Cedar St, Someplace, USA', '+1999888777'), (7, 'David Martinez', '234 Birch St, Elsewhere, USA', '+1666333444'), (8, 'Jennifer Taylor', '876 Walnut St, Nowhere, USA', '+1888999000'), (9, 'James Rodriguez', '543 Spruce St, Anywhere, USA', '+1777666555'), (10, 'Emma Thomas', '210 Cherry St, Somewhere, USA', '+1222111333')]"

In [3]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [4]:
agent_executor.invoke(
    "List the total bill per customer. Which customer has the most in Payments?"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with ``


Billing, Complaints, Customers, Devices, Employees, Locations, Messages, Notifications, Orders, Payments, Plans, Repairs, Services, Subscriptions, Usage
Invoking: `sql_db_schema` with `{'table_names': 'Customers, Payments'}`



CREATE TABLE "Customers" (
	customer_id INTEGER, 
	name TEXT, 
	address TEXT, 
	contact_number TEXT, 
	PRIMARY KEY (customer_id)
)

/*
3 rows from Customers table:
customer_id	name	address	contact_number
1	John Doe	123 Main St, Anytown, USA	+1234567890
2	Alice Smith	456 Elm St, Otherville, USA	+1987654321
3	Bob Johnson	789 Oak St, Anycity, USA	+1122334455
*/


CREATE TABLE "Payments" (
	payment_id INTEGER, 
	customer_id INTEGER, 
	payment_date DATE, 
	amount REAL, 
	payment_method TEXT, 
	PRIMARY KEY (payment_id), 
	FOREIGN KEY(customer_id) REFERENCES "Customers" (customer_id)
)

/*
3 rows from Payments table:
payment_id	customer_id	payment_date	amount	payment_method
1	1	2024-

{'input': 'List the total bill per customer. Which customer has the most in Payments?',
 'output': 'The total bill per customer is as follows:\n1. Sarah Davis - $239.97\n2. James Rodriguez - $239.97\n3. Bob Johnson - $239.97\n4. Michael Wilson - $149.97\n5. Jennifer Taylor - $149.97\n6. Alice Smith - $149.97\n7. John Doe - $89.97\n8. Emma Thomas - $89.97\n9. Emily Brown - $89.97\n10. David Martinez - $89.97\n\nThe customers with the most payments are Sarah Davis, James Rodriguez, and Bob Johnson, each with a total bill of $239.97.'}

In [5]:
agent_executor.invoke("Which Plan is the most popular?")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with ``


Billing, Complaints, Customers, Devices, Employees, Locations, Messages, Notifications, Orders, Payments, Plans, Repairs, Services, Subscriptions, Usage
Invoking: `sql_db_schema` with `{'table_names': 'Subscriptions,Plans'}`



CREATE TABLE "Plans" (
	plan_id INTEGER, 
	plan_name TEXT, 
	description TEXT, 
	price REAL, 
	PRIMARY KEY (plan_id)
)

/*
3 rows from Plans table:
plan_id	plan_name	description	price
1	Basic	Basic plan with limited data and minutes	29.99
2	Standard	Standard plan with moderate data and minutes	49.99
3	Premium	Premium plan with unlimited data and minutes	79.99
*/


CREATE TABLE "Subscriptions" (
	subscription_id INTEGER, 
	customer_id INTEGER, 
	plan_id INTEGER, 
	service_id INTEGER, 
	start_date DATE, 
	end_date DATE, 
	PRIMARY KEY (subscription_id), 
	FOREIGN KEY(customer_id) REFERENCES "Customers" (customer_id), 
	FOREIGN KEY(plan_id) REFERENCES "Plans" (plan_id), 
	FOREIGN 

{'input': 'Which Plan is the most popular?',
 'output': 'The most popular plan is the "Basic" plan, with 4 subscriptions.'}

In [37]:
examples = [
    {"input": "List all subscription plans.", "query": "SELECT * FROM Plans;"},
    {"input": "What is the name for plan_id 1?", "query": "SELECT plan_name FROM Plans where plan_id=1;"},
    {
        "input": "Find all customers for the plan 'Basic'.",
        "query": """
                    SELECT * FROM Customers
                    WHERE customer_id IN (
                        SELECT customer_id FROM Subscriptions
                        WHERE plan_id = (
                            SELECT plan_id FROM Plans WHERE plan_name = 'Basic'
                            );
                    );
                """,
    },
    {
        "input": "List all locations with 'International Roaming' service active.",
        "query": """
            SELECT DISTINCT L.location_name
            FROM Locations L
            JOIN Devices D ON L.location_id = D.location_id
            JOIN Subscriptions S ON D.customer_id = S.customer_id
            JOIN Services SER ON S.service_id = SER.service_id
            WHERE SER.service_name = 'International Roaming';
            """,
    },
    {
        "input": "Find the total duration of all the calls made in each month.",
        "query": """
        SELECT
            strftime('%Y-%m', M.sent_date) AS month,
            SUM(U.duration) AS total_duration
        FROM
            Messages M
        JOIN
            Usage U ON M.sender_id = U.customer_id
        GROUP BY
            month
        ORDER BY
            month;
        """,
    },
    {
        "input": "Who are the top 5 customers by total payments?",
        "query": """SELECT
                    customer_id,
                    SUM(amount) AS total_payments
                FROM
                    Payments
                GROUP BY
                    customer_id
                ORDER BY
                    total_payments DESC
                LIMIT 5;
            """,
    },
    {
        "input": "How many employees are there",
        "query": """SELECT COUNT(*) FROM 'Employees'""",
    },
    {
        "input": "What is the plan of customer Bob?",
        "query": """SELECT
                    P.plan_name
                FROM
                    Customers C
                JOIN
                    Subscriptions S ON C.customer_id = S.customer_id
                JOIN
                    Plans P ON S.plan_id = P.plan_id
                WHERE
                    C.name = 'Bob';
                    """
    },
]

In [33]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=5,
    input_keys=["input"],
)

from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix="",
)

full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [38]:
agent = create_sql_agent(
    llm=llm,
    db=db,
    prompt=full_prompt,
    verbose=False, #Change this to True if you want agent execution details
    agent_type="openai-tools",
)
agent.invoke({"input": "Can you name the customer with Basic Subscriptions?"})

{'input': 'Can you name the customer with Basic Subscriptions?',
 'output': 'The customers with Basic subscriptions are John Doe, Emily Brown, David Martinez, and Emma Thomas.'}

In [81]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


customer_names = query_as_list(db, "SELECT name FROM Customers")
plan_details = query_as_list(db, "SELECT description FROM Plans")


from langchain.agents.agent_toolkits import create_retriever_tool

vector_db = FAISS.from_texts(customer_names + plan_details, OpenAIEmbeddings())
retriever = vector_db.as_retriever(search_kwargs={"k": 1})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is \
valid proper nouns. Use the noun most similar to the search."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

table_names = db.get_usable_table_names()

system = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If you need to filter on a proper noun like customer name or a plan description, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool!

You have access to the following tables: {table_names}.

If the question does not seem related to the database, just return "I don't know" as the answer."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}"), MessagesPlaceholder("agent_scratchpad")]
)

agent = create_sql_agent(
    llm=llm,
    db=db,
    extra_tools=[retriever_tool],
    prompt=prompt,
    agent_type="openai-tools",
    verbose=True,
)

In [61]:
agent.invoke({"input": "Is there a customer with name Bobb?"})



> Entering new SQL Agent Executor chain...

Invoking: `search_proper_nouns` with `{'query': 'Bobb'}`


Bob Johnson
Invoking: `sql_db_query` with `SELECT * FROM Customers WHERE name = 'Bob Johnson'`


[(3, 'Bob Johnson', '789 Oak St, Anycity, USA', '+1122334455')]Yes, there is a customer with the name Bob Johnson. Their address is 789 Oak St, Anycity, USA and their phone number is +1122334455.

> Finished chain.


{'input': 'Is there a customer with name Bobb?',
 'output': 'Yes, there is a customer with the name Bob Johnson. Their address is 789 Oak St, Anycity, USA and their phone number is +1122334455.'}

In [82]:
agent.invoke({"input": "Which plan has unlimited data?"})



> Entering new SQL Agent Executor chain...

Invoking: `search_proper_nouns` with `{'query': 'unlimited data'}`


Premium plan with unlimited data and minutes
Invoking: `sql_db_query` with `SELECT * FROM Plans WHERE description = 'Premium plan with unlimited data and minutes'`


[(3, 'Premium', 'Premium plan with unlimited data and minutes', 79.99)]The plan that has unlimited data is the "Premium" plan, which costs $79.99.

> Finished chain.


{'input': 'Which plan has unlimited data?',
 'output': 'The plan that has unlimited data is the "Premium" plan, which costs $79.99.'}